# Analisi delle varianti di RProp per la classificazione del dataset MNIST

## Introduzione

In questo quaderno, esploreremo le prestazioni di diverse varianti dell'algoritmo di ottimizzazione RProp (Resilient Backpropagation) applicate alla classificazione del famoso dataset MNIST. Il dataset MNIST è un insieme di immagini di cifre scritte a mano, ampiamente utilizzato come benchmark nell'ambito dell'apprendimento automatico e del riconoscimento di pattern.

## Obiettivo

L'obiettivo principale di questo studio è valutare le prestazioni di diverse varianti di RProp nell'addestramento di reti neurali artificiali per la classificazione del dataset MNIST. In particolare, esploreremo:

- RProp standard (RProp-)
- RProp con backtracking dei pesi (RProp+)
- Improved RProp (iRProp-)
- Improved RProp con backtracking dei pesi (iRProp+)

Per ciascuna variante di RProp, addestreremo una rete neurale artificiale originaria e ne valuteremo le prestazioni attraverso grafici che mostrano l'andamento dell'errore e della precisione sia sul set di addestramento che sul set di validazione. Infine, testeremo le reti addestrate su un set di test e visualizzeremo le probabilità predette per alcune immagini del dataset.

## Contenuti

1. Estrazione e preparazione del dataset MNIST
2. Creazione della rete neurale originaria
3. Addestramento delle reti neurali con diverse varianti di RProp
4. Valutazione delle prestazioni attraverso grafici
5. Test delle reti addestrate su nuove immagini
6. Conclusioni e osservazioni finali


In [1]:
from uninaannpy import neural_network as nn
from uninaannpy import error_functions as ef
from uninaannpy import activation_functions as af
from uninaannpy import datasets as ds
from uninaannpy import utility as ut

import numpy as np
import pandas as pd

# Lettura csv con pandas
train_data = pd.read_csv('data/mnist_train.csv')
test_data = pd.read_csv('data/mnist_test.csv')

# Ottenimento array per il training e il testing
train_array = np.array(train_data)
test_array = np.array(test_data)

# Estrapoliamo il numero di righe e di colonne di train e test
train_rows, train_cols = train_array.shape 
test_rows, test_cols = test_array.shape  

# Mescola casualmente i dati prima di suddividerli in set di sviluppo e training
np.random.shuffle(train_array)  
#Calcolo percentuale dati per il validation set
validation_percentage = 0.20
val_index = int(np.ceil(train_rows * validation_percentage))

# Distribuzione training, validation e test set
train_in, train_labels = ds.get_mnist_training(train_array, train_rows, train_cols, val_index)
validation_in, validation_labels = ds.get_mnist_validation(train_array, train_cols, val_index)
test_in, test_labels = ds.get_mnist_testing(test_array, test_cols, test_rows)

# Creazione rete neurale

# Assegnazione funzioni di attivazione per ogni strato della rete
hidden_activation_functions = [af.tanh]

# Assegnazione numero di neuroni per ogni strato
hidden_layers = [200]
#Assegnazione della funzione di attivazione e di errore per l'ultimo strato della rete
output_activation_function = af.identity
error_function = ef.cross_entropy_softmax

# Estrazione input e target
input_layer_size = train_in.shape[0]
output_layer_size = train_labels.shape[0]
net = nn.NeuralNetwork(hidden_activation_functions, output_activation_function, error_function,
                input_layer_size, hidden_layers, output_layer_size)

# Copia la rete principale in quattro per il training
std_training_net = net.duplicate_network()
plus_training_net = net.duplicate_network()
istd_training_net = net.duplicate_network()
iplus_training_net = net.duplicate_network()

std_training_net.get_net_structure()

Numero di strati nascosti: 1
Dimensione dell'input: 784
Dimensione dell'output: 10
Neuroni negli strati nascosti: 200
Funzioni di attivazione: tanh, identity
Funzione di errore: cross_entropy_softmax


In [2]:
from uninaannpy.neural_network import RPropType

# Batch training RProp standard
std_train_err, std_validation_err, std_train_accuracy, std_validation_accuracy = std_training_net.train_neural_network(train_in, train_labels, validation_in, validation_labels, epochs=40, learning_rate=0.00001, rprop_type=RPropType.STANDARD)


Epoca: 0/40    rProp utilizzata: RPropType.STANDARD
    Training Accuracy: 9.13561%,
    Validation Accuracy: 9.39245%

Epoca: 1/40   RProp utilizzata: RPropType.STANDARD
    Training Accuracy: 33.50695%,
    Validation Accuracy: 33.00275%

Epoca: 2/40   RProp utilizzata: RPropType.STANDARD
    Training Accuracy: 35.43615%,
    Validation Accuracy: 35.94466%

Epoca: 3/40   RProp utilizzata: RPropType.STANDARD
    Training Accuracy: 38.94873%,
    Validation Accuracy: 38.34486%

Epoca: 4/40   RProp utilizzata: RPropType.STANDARD
    Training Accuracy: 26.50055%,
    Validation Accuracy: 26.62722%

Epoca: 5/40   RProp utilizzata: RPropType.STANDARD
    Training Accuracy: 40.72793%,
    Validation Accuracy: 41.12009%

Epoca: 6/40   RProp utilizzata: RPropType.STANDARD
    Training Accuracy: 56.01158%,
    Validation Accuracy: 55.5963%

Epoca: 7/40   RProp utilizzata: RPropType.STANDARD
    Training Accuracy: 71.68483%,
    Validation Accuracy: 71.46429%

Epoca: 8/40   RProp utilizzata: R

In [3]:
from uninaannpy.neural_network import RPropType

# Batch training RProp con weight-backtracking
plus_train_err, plus_validation_err, plus_train_accuracy, plus_validation_accuracy = plus_training_net.train_neural_network(train_in, train_labels, validation_in, validation_labels, epochs=40, learning_rate=0.00001, rprop_type=RPropType.RPROP_PLUS)


Epoca: 0/40    rProp utilizzata: RPropType.RPROP_PLUS
    Training Accuracy: 9.13561%,
    Validation Accuracy: 9.39245%

Epoca: 1/40   RProp utilizzata: RPropType.RPROP_PLUS
    Training Accuracy: 33.50695%,
    Validation Accuracy: 33.00275%

Epoca: 2/40   RProp utilizzata: RPropType.RPROP_PLUS
    Training Accuracy: 51.66983%,
    Validation Accuracy: 52.17935%

Epoca: 3/40   RProp utilizzata: RPropType.RPROP_PLUS
    Training Accuracy: 28.87977%,
    Validation Accuracy: 28.01067%

Epoca: 4/40   RProp utilizzata: RPropType.RPROP_PLUS
    Training Accuracy: 35.55907%,
    Validation Accuracy: 36.16968%

Epoca: 5/40   RProp utilizzata: RPropType.RPROP_PLUS
    Training Accuracy: 37.32369%,
    Validation Accuracy: 37.91149%

Epoca: 6/40   RProp utilizzata: RPropType.RPROP_PLUS
    Training Accuracy: 43.21132%,
    Validation Accuracy: 42.96191%

Epoca: 7/40   RProp utilizzata: RPropType.RPROP_PLUS
    Training Accuracy: 46.92181%,
    Validation Accuracy: 47.46229%

Epoca: 8/40   RP

In [ ]:
from uninaannpy.neural_network import RPropType

# Batch training improved RProp con weight-backtracking
istd_train_err, istd_validation_err, istd_train_accuracy, istd_validation_accuracy = istd_training_net.train_neural_network(train_in, train_labels, validation_in, validation_labels, epochs=40, learning_rate=0.00001, rprop_type=RPropType.IRPROP)


Epoca: 0/40    rProp utilizzata: RPropType.IRPROP
    Training Accuracy: 9.13561%,
    Validation Accuracy: 9.39245%

Epoca: 1/40   RProp utilizzata: RPropType.IRPROP
    Training Accuracy: 33.50695%,
    Validation Accuracy: 33.00275%

Epoca: 2/40   RProp utilizzata: RPropType.IRPROP
    Training Accuracy: 35.43615%,
    Validation Accuracy: 35.94466%

Epoca: 3/40   RProp utilizzata: RPropType.IRPROP
    Training Accuracy: 38.80081%,
    Validation Accuracy: 37.81982%


In [ ]:
from uninaannpy.neural_network import RPropType

# Batch training improved RProp
iplus_train_err, iplus_validation_err, iplus_train_accuracy, iplus_validation_accuracy = iplus_training_net.train_neural_network(train_in, train_labels, validation_in, validation_labels, epochs=40, learning_rate=0.00001, rprop_type=RPropType.IRPROP_PLUS)

In [ ]:
from matplotlib import pyplot as plt

# Stampa grafico dell'errore del training
plt.figure()
plt.plot(std_train_err, 'b', label='RPROP-')
plt.plot(plus_train_err, 'r', label='RPROP+')
plt.plot(istd_train_err, 'y', label='IRPROP-')
plt.plot(iplus_train_err, 'g', label='IRPROP+')
plt.xlabel('Epoche')
plt.ylabel('Errori')
plt.legend()
plt.show()

In [ ]:
from matplotlib import pyplot as plt

# Stampa grafico dell'errore del validation
plt.figure()
plt.plot(std_validation_err, 'b', label='RPROP-')
plt.plot(plus_validation_err, 'r', label='RPROP+')
plt.plot(istd_validation_err, 'y', label='IRPROP-')
plt.plot(iplus_validation_err, 'g', label='IRPROP+')
plt.xlabel('Epoche')
plt.ylabel('Errori')
plt.legend()
plt.show()

In [ ]:
from matplotlib import pyplot as plt

# Stampa grafico della accuracy del training
plt.figure()
plt.plot(std_train_accuracy, 'b', label='RPROP-')
plt.plot(plus_train_accuracy, 'r', label='RPROP+')
plt.plot(istd_train_accuracy, 'y', label='IRPROP-')
plt.plot(iplus_train_accuracy, 'g', label='IRPROP+')
plt.xlabel('Epoche')
plt.ylabel('Precisione')
plt.legend()
plt.show()

In [ ]:
from matplotlib import pyplot as plt

# Stampa grafico della accuracy del validation
plt.figure()
plt.plot(std_validation_accuracy, 'b', label='RPROP-')
plt.plot(plus_validation_accuracy, 'r', label='RPROP+')
plt.plot(istd_validation_accuracy, 'y', label='IRPROP-')
plt.plot(iplus_validation_accuracy, 'g', label='IRPROP+')
plt.xlabel('Epoche')
plt.ylabel('Precisione')
plt.legend()
plt.show()

In [ ]:
# Stampa accuracy per training e test set

std_training_net.print_accuracies('Test RProp-', test_in, test_labels, train_in, train_labels)

In [ ]:
# Stampa accuracy per training e test set

plus_training_net.print_accuracies('Test RProp+', test_in, test_labels, train_in, train_labels)

In [ ]:
# Stampa accuracy per training e test set

istd_training_net.print_accuracies('Test iRProp-', test_in, test_labels, train_in, train_labels)

In [ ]:
# Stampa accuracy per training e test set

iplus_training_net.print_accuracies('Test iRProp+', test_in, test_labels, train_in, train_labels)

In [ ]:
# Predizione rete senza addestramento e rete addestrata
image = 47993

print('Test RProp-')
ut.test_prediction(net, std_training_net, image, train_in)

In [ ]:
# Predizione rete senza addestramento e rete addestrata
image = 47993

print('Test RProp+')
ut.test_prediction(net, plus_training_net, image, train_in)

In [ ]:
# Predizione rete senza addestramento e rete addestrata
image = 47993

print('Test iRProp-')
ut.test_prediction(net, istd_training_net, image, train_in)

In [ ]:
# Predizione rete senza addestramento e rete addestrata
image = 47993

print('Test iRProp+')
ut.test_prediction(net, iplus_training_net, image, train_in)